## Always run thic CELL below
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPieplinefactory

In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = False
p.active_model = 11 # 10=titanic , 11=Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()


Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_split_and_register.py
File to EDIT (step: TRAIN_MANUAL): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_manual.py
File to EDIT a lot (reference in step-scripts Custom code): ../../../2_A_aml_pipeline/4_inference/batch/M11/your_code/your_custom_code.py

 ---- WHAT model to

# IN_2_GOLD_TRAIN_MANUAL

In [2]:
# Override default CURATED environment = "AzureML-AutoML-DNN", "AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"
p_factory.use_curated_automl_environment = True
p_factory.environment_name = "AzureML-AutoML" # Training[ "AzureML-AutoML", "AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"]  Inference["AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference","AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"]

In [3]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_TRAIN_MANUAL)

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False
Using GEN2 as Datastore
ESML will auto-create a compute...
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster p02-m11weu-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = p02-m11weu-dev
Reusing existing compute...
create_gold_train_step: inference_mode=False
ESML-train_path_out = 
Tryin

In [4]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

Execute_pipeline (scoring): Inference_mode: 0
-Scoring data, default value 1000-01-01 10:35:01.243860
Created step IN 2 SILVER - ds01_diabetes [3ed2c01c][f8e9dbdd-f7bb-405d-bb47-5814910aaaf1], (This step will run and generate new outputs)Created step IN 2 SILVER - ds02_other [112ebeb6][c6a7156d-6f83-4a94-92b7-8c438be95138], (This step will run and generate new outputs)

Created step SILVER MERGED 2 GOLD [a6e68140][d441ae09-147a-42e2-b0c1-8ab184253527], (This step will run and generate new outputs)
Created step SPLIT AND REGISTER datasets [e455000b][c2d27648-b468-4bbb-bce7-de3909e41dbb], (This step will run and generate new outputs)
Created step TRAIN in  [dev], COMPARE & REGISTER model in [dev] & PROMOTE to [test] [4271abea][ba2efead-345a-47e4-8a2d-7aee0801f0f0], (This step will run and generate new outputs)
Submitted PipelineRun 7fdb8c97-66d8-4f5a-9d5b-4509d3b4adc2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7fdb8c97-66d8-4f5a-9d5b-4509d3b4adc2?wsid=/subscriptions

'Finished'

In [4]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_6
pub_pipe.id 01274209-95e8-4ffb-9b95-9bfb1c865b28
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_1
pub_pipe.id cb6e1ef3-ddf2-483c-964e-bc6ba02c610f
pub_pipe.name 10_titanic_model_clas_pipe_IN_2_GOLD_SCORING_EP_1
pub_pipe.id 6c3c51e9-df26-4373-ae37-0a31a3616ec0


# Get info - to use  in Azure data factory: `published_pipeline.id` (if private Azure ML workspace)

In [5]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter
- Endpoint ID
Endpoint ID:  6c3c51e9-df26-4373-ae37-0a31a3616ec0
Endpoint Name:  10_titanic_model_clas_pipe_IN_2_GOLD_SCORING_EP_1
Experiment name:  10_titanic_model_clas_pipe_IN_2_GOLD_SCORING
In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.
-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)


'0a3ffd65-f1e2-4705-8fd0-681e4d16fbe4'